In [3]:
from pathlib import Path
import pandas as pd
import json
from itertools import islice

In [4]:
occ_rename_dict = {
    'OPM SERIES NUMBER (December 2018)': 'opm_occ_code',
    'OPM SERIES TITLE\xa0\n(December 2018)': 'opm_occ_title',
    '2018 \nSOC CODE': '2018_soc_code',
    '2014-2018 EEO TABULATION (CENSUS) CODE': 'census_occ_code',
    '2014-2018 EEO TABULATION (CENSUS) OCCUPATION TITLE': 'census_occ_title'
}

In [5]:
# Load OPM occupation code to SOC code crosswalk
opm_occ_df = pd.read_excel('../raw_data/MD715-Census Occupation Crosswalk 2022Feb23.xlsx', header = 2, dtype = 'str')
opm_occ_df.rename(columns = occ_rename_dict, inplace = True)
opm_occ_df = opm_occ_df[list(occ_rename_dict.values())]

In [6]:
# Remove trailing #, which indicates deviations from documentation of the 2014-2018 EEO Tabulation
for col in opm_occ_df.columns:
    opm_occ_df[col] = opm_occ_df[col].str.replace('#', '')

In [7]:
dutystation_rename_dict = {
    'Code': 'duty_station_code',
    'CBSA': 'cbsa',
    'CSA': 'csa',
    'City': 'city',
    'County': 'county',
    'State': 'state',
    'Country': 'country'
}

In [8]:
# Load duty station to county crosswalk file
opm_dutystation_df = pd.read_excel('../raw_data/opm_dutystation.xlsx', dtype = 'str')
opm_dutystation_df.rename(columns = dutystation_rename_dict, inplace = True)
opm_dutystation_df = opm_dutystation_df[list(dutystation_rename_dict.values())]

# Pad duty station code to 9 characters with 0s on the left
opm_dutystation_df['duty_station_code'] = opm_dutystation_df['duty_station_code'].str.pad(9, side = 'left', fillchar = '0')

# Remove duty stations not in a county
opm_dutystation_df = opm_dutystation_df[~opm_dutystation_df['county'].isna()]
opm_dutystation_df['msa_code'] = pd.Series(dtype = 'str') # Add column for MSA code

In [9]:
# Load QCEW county-MSA crosswalk
qcew_df = pd.read_excel('../raw_data/qcew-county-msa-csa-crosswalk-xlsx.xlsx')
#qcew_df = qcew_df[~qcew_df['MSA Code'].isna()]
qcew_df.loc[qcew_df['MSA Code'].isna(), 'MSA Code'] = 'matched'

# Split conty title into county and state columns, capitalized to merge with OPM dutystation code file
qcew_df['county'] = qcew_df['County Title'].str.split(', ').str[0]
qcew_df['state'] = qcew_df['County Title'].str.split(', ').str[1]

qcew_df['state'] = qcew_df['state'].str.upper()
qcew_df.loc[qcew_df['state'].isna(), 'state'] = 'DISTRICT OF COLUMBIA' # DC has blank state
qcew_df.loc[qcew_df['state'] == 'AK', 'state'] = 'ALASKA' # One entry has AK instead of Alaska

qcew_df['county'] = qcew_df['county'].str.upper()

In [10]:
# List of states that are in both QCEW and OPM (50 states + DC)
qcew_state_list = list(qcew_df['state'].unique())
opm_dutystation_state_list = list(opm_dutystation_df['state'].unique())
state_list = list(set(qcew_state_list) & set(opm_dutystation_state_list))

In [11]:
# Iterate over states and counties to add MSA code to OPM dutystation df
for state_name in state_list:
    county_list = list(opm_dutystation_df['county'][opm_dutystation_df['state'] == state_name].unique())
    for county_name in county_list:

        qcew_county_name = county_name.replace('SAINT ', 'ST. ')
        qcew_county_name = qcew_county_name.replace('DE KALB', 'DEKALB')
        qcew_county_name = qcew_county_name.replace('DU PAGE', 'DUPAGE')
        qcew_county_name = qcew_county_name.replace('LA PORTE', 'LAPORTE')
        qcew_county_name = qcew_county_name.replace('DONA ANA', 'DOÑA ANA')
        qcew_county_name = qcew_county_name.replace('LA MOURE', 'LAMOURE')
        qcew_county_name = qcew_county_name.replace('SPAULDING', 'SPALDING') # Typo in OPM county name
        qcew_county_name = qcew_county_name.replace('O BRIEN', 'OBRIEN')
        qcew_county_name = qcew_county_name.replace('STE GENEVIEVE', 'STE. GENEVIEVE')
        # La Salle has a space in Texas, but not in other states
        if state_name != 'TEXAS':
            qcew_county_name = qcew_county_name.replace('LA SALLE', 'LASALLE')
        # De Soto has a space in Louisiana, but not in other states
        if state_name != 'LOUISIANA':
            qcew_county_name = qcew_county_name.replace('DE SOTO', 'DESOTO')

        try:
            msa_code = qcew_df[(qcew_df['state'] == state_name) & (qcew_df['county'].str.contains(qcew_county_name))]['MSA Code'].iloc[0]
        except:
            continue

        opm_dutystation_df.loc[(opm_dutystation_df['state'] == state_name) & (opm_dutystation_df['county'] == county_name), 'msa_code'] = msa_code # Assign MSA code

In [12]:
# All counties matched, with counties outside MSAs assigned an MSA code of 'matched'
sum(opm_dutystation_df['msa_code'].isna())

0

In [13]:
# Load dicts of paths of binaries
with open('../code_output/opm_nondod_status_pre2014_feather_path_dict.json', 'r') as infile:
    opm_nondod_status_feather_path_dict = json.load(infile)

with open('../code_output/opm_dod_status_pre2014_feather_path_dict.json', 'r') as infile:
    opm_dod_status_feather_path_dict = json.load(infile)

In [14]:
opm_collapsed_feather_path_dict = {}
binary_path = Path('../cleaned_binaries/')

In [335]:
for year, qtr_dict in opm_nondod_status_feather_path_dict.items():
    year_df = pd.DataFrame()

    if int(year) < 1990:
        continue
    
    print(f'Collapsing year {year}.')

    for qtr, file_path in qtr_dict.items():
        df = pd.read_feather(Path(file_path))
        year_df = pd.concat([year_df, df])

    year_df = year_df[['Pseudo-ID', 'Employee Name', 'Duty Station', 'Occupation', 'Adjusted Basic Pay']] # Keep only variables we want
    
    # Add CBSA and SOC codes
    merged_df = year_df.merge(opm_occ_df, how = 'inner', left_on = ['Occupation'], right_on = ['opm_occ_code'])
    merged_df = merged_df.merge(opm_dutystation_df, how = 'inner', left_on = ['Duty Station'], right_on = ['duty_station_code'])

    # Keep only 1 observation per worker in a given year
    merged_df = merged_df[merged_df['msa_code'] != 'matched'] # Drop people who do not have an MSA code
    merged_df.drop_duplicates(subset = ['Pseudo-ID'])

    # Collapse by SOC code and CBSA
    collapsed_df = merged_df.groupby(by = ['2018_soc_code', 'msa_code']).agg({
        'Pseudo-ID': 'count',
        'Adjusted Basic Pay': 'mean',
        'opm_occ_code': 'first',
        'opm_occ_title': 'first',
        'census_occ_code': 'first',
        'census_occ_title': 'first',
        'county': 'first',
        'state': 'first',
    })

    # Cleap up collapsed df and export
    collapsed_df.reset_index(inplace = True)
    collapsed_df.rename(columns = {'Pseudo-ID': 'opm_n_emp', 'Adjusted Basic Pay': 'mean_abp'}, inplace = True)
    
    target_path = Path(binary_path).joinpath(f'opm_collapsed_{year}.feather')
    collapsed_df.to_feather(target_path)

    opm_collapsed_feather_path_dict[year] = str(target_path)

Collapsing year 1990.
Collapsing year 1991.
Collapsing year 1992.
Collapsing year 1993.
Collapsing year 1994.
Collapsing year 1995.
Collapsing year 1996.
Collapsing year 1997.
Collapsing year 1998.
Collapsing year 1999.
Collapsing year 2000.
Collapsing year 2001.
Collapsing year 2002.
Collapsing year 2003.
Collapsing year 2004.
Collapsing year 2005.
Collapsing year 2006.
Collapsing year 2007.
Collapsing year 2008.
Collapsing year 2009.
Collapsing year 2010.
Collapsing year 2011.
Collapsing year 2012.
Collapsing year 2013.
Collapsing year 2014.


In [336]:
# Save dictionaries for paths of binaries
with open('../code_output/opm_collapsed_pre2014_feather_path_dict.json', 'w') as outfile:
    json.dump(opm_collapsed_feather_path_dict, outfile, indent = 4)